![Insper](https://github.com/danielscarvalho/Insper-DS-Dicas/blob/master/Insper-Logo.png?raw=true)

# Insper Pós-Graduação
## Programa Avançado em Data Science e Decisão [»](https://www.insper.edu.br/pos-graduacao/programas-avancados/programa-avancado-em-data-science-e-decisao/)


# Atividade Integradora
## Setup

### Dependencias

In [ ]:
import pandas as pd
import numpy as np
from dfply import *
import altair as alt
import missingno as msno
from ydata_profiling import ProfileReport
import matplotlib
import matplotlib.pyplot as plt 
import statsmodels.api as sm
import math

### Carregamento dos Dados

In [ ]:
data = pd.read_csv("cs_bisnode_panel.csv")
dicionario_de_dados_0 = pd.read_excel("bisnode_variable_names.xls", header=4)

In [ ]:
data.head()

## Limpeza dos Dados

### Dicionário de dados - Limpeza

Ao carregar o dicionário de dados a primeira coluna pega seu nome da quarta linha da tabela (argumento `header=4` acima). As outras três colunas são nomeadas abaixo.

In [ ]:
dicionario_de_dados_1 = dicionario_de_dados_0.rename({'Unnamed: 1': 'description',
                                                      'Unnamed: 2': 'type',
                                                      'Unnamed: 3': 'footnote'},
                                                     axis=1)

In [ ]:
dicionario_de_dados_1.columns

Então, retiramos as linhas não relevantes para a análise, incluindo linhas totalmente em branco e uma linha com informação de versão da base de dados: 
 - `v 0.92. 2021-02-04`

In [ ]:
dicionario_de_dados = dicionario_de_dados_1\
                       .drop(index=54)\
                       .dropna(how="all")\
                       .reset_index()\
                       .drop('index', axis='columns')

In [ ]:
dicionario_de_dados.sample(5)

## Dados
### Removendo colunas específicas:

In [ ]:
columns_to_remove = ['COGS', 'finished_prod', 'net_dom_sales', 'net_exp_sales', 'wages', 'D']

data.drop(columns=columns_to_remove, inplace=True)

data.columns

### Removendo dados do ano 2016:
---

Registros que começaram ou terminaram (ocorreram em) 2016 são removidos do conjunto.

In [ ]:
data = data[data["year"]!=2016]
data["year"].unique()

Dados destas colunas precisam estar em formato de datetime para serem operados corretamente, então vamos converte-los.

In [ ]:
colunas_data = ["begin", "end", "founded_date", "exit_date"]

data.dtypes[colunas_data]

In [ ]:
for column in colunas_data:
    data[column] = pd.to_datetime(data[column], format="%Y-%m-%d")

data[colunas_data].dtypes

### Missing data:
---

Vamos verificar dados faltando do banco de dados:

In [ ]:
def show_missing(df):
    """Return a Pandas dataframe describing the contents of a source dataframe including missing values."""
    
    variables = []
    dtypes = []
    count = []
    unique = []
    missing = []
    pc_missing = []
    
    for item in df.columns:
        variables.append(item)
        dtypes.append(df[item].dtype)
        count.append(len(df[item]))
        unique.append(len(df[item].unique()))
        missing.append(df[item].isna().sum())
        pc_missing.append(round((df[item].isna().sum() / len(df[item])) * 100, 2))

    output = pd.DataFrame({
        'variable': variables, 
        'dtype': dtypes,
        'count': count,
        'unique': unique,
        'missing': missing, 
        'pc_missing': pc_missing
    })    
        
    return output

Verificando dados com maior falta de informações:

In [ ]:
missing_data = show_missing(data).sort_values("pc_missing", ascending=False, ignore_index = True)

index_full_data = list(missing_data[missing_data["missing"]==0].index)

missing_data.drop(labels=index_full_data, axis="index", inplace=True)

missing_data

Vamos iniciar tratando os dados que estejam com menos de 5% de dados faltando.

Podemos considerar que pela quantidade de dados estes casos são insignificantes no escopo total então vamos apenas remover estes.

(apenas para as colunas com baixa falta de dados ">5%")

In [ ]:
removable_na_columns = list(missing_data["variable"][missing_data["pc_missing"]<5.0])

data.dropna(subset=removable_na_columns, ignore_index=True, inplace=True)

In [ ]:
missing_val_columns = missing_data["variable"][missing_data["pc_missing"]>0]

In [ ]:
msno.bar(data[missing_val_columns], figsize=(16, 4))

In [ ]:
missing_data_2 = show_missing(data).sort_values("pc_missing", ascending=False, ignore_index = True)

index_full_data_2 = list(missing_data_2[missing_data_2["missing"]==0].index)

missing_data_2.drop(labels=index_full_data_2, axis="index", inplace=True)

missing_data_2

### Criando coluna para Variável Dependente:
---

- Vamos operar com o conceito de "atividade", empresas que não tiveram atividade por mais de 2 anos são consideradas "inativas".

Obs.: ativa = 0, inativa = 1

In [ ]:
# Sort the DataFrame by company and year
data.sort_values(by=['comp_id', 'year'], inplace=True)

# Create shifted columns to check sales in the following 2 years
conditions_x1 = [((data['comp_id'] == data['comp_id'].shift(-1)) &
                  (data['year'] == data['year'].shift(-1) - 1)),

                 ((data['comp_id'] != data['comp_id'].shift(-1)) |
                  data['year'] != data['year'].shift(-1))]

values_x1 = [data['sales'].shift(-1),
             np.nan]

data['sales_x1'] = np.select(conditions_x1, values_x1)


conditions_x2 = [
    ((data['comp_id'] == data['comp_id'].shift(-1)) &   
     (data['year'] == data['year'].shift(-1) - 2)),

    ((data['comp_id'] == data['comp_id'].shift(-2)) &
     (data['year'] == data['year'].shift(-2) - 2)),

    True
]

values_x2 = [data['sales'].shift(-1),
             data['sales'].shift(-2),
             np.nan]

data['sales_x2'] = np.select(conditions_x2, values_x2)


# Create a condition to identify companies that ceased to operate 
# (sem vendas por mais de 2 anos)
condition = ((data["sales_x1"]==0) & (data["sales_x2"]==0))


# Create a new 'dependente' column with 1 for ceased companies and 0 otherwise
data['fechado'] = condition.astype(int)


Vamos conferir os valores da variavel dependente para as duas primeiras empresas do dataframe:

In [ ]:
# lista de colunas para avaliação:
check_list=["comp_id", "year", "exit_year","sales", "sales_x1", "sales_x2", "fechado"]

filtro = data[check_list]
filtro[filtro["comp_id"] == 1001034]

In [ ]:
data[check_list].describe()

Vemos que temos dados de venda com erros, onde há valores negativos para vendas, vamos tratar estes dados substituindo valores negativos por nulos (valor = 0)

In [ ]:
# Remodelando os dados de vendas para corrigir os erros:

for x in ["sales", "sales_x1", "sales_x2"]:
    data[x] = np.where(data[x]<0, 0, data[x])

data['fechado'] = condition.astype(int)

In [ ]:
data[check_list].describe()

Corrigimos os valores de vendas negativas, e assim também de dependentes que pudessem estar sendo afetados.

Vamos agora:
 
- tratar por fim os casos em que não há informações de venda para um próximo ano (por não haver um próximo ano;

- criar uma coluna de vendas em "Log" para tratar a assimetria dos dados.

Vamos averiguar agora os anos em que as empresas tiveram atividade e inatividade:

In [ ]:
data_grouped = data.groupby('comp_id')

#Contando anos de acompanhamento
comp_years = data_grouped['year'].count()

#Contando anos de "atividade"
active_years = data_grouped.apply(lambda group: (group['sales'] > 0).sum())

#contando anos sem vendas
years_no_sales = data_grouped.apply(lambda group: (group['sales'] == 0).sum())

#Contando anos de "inatividade"
inative_years = data_grouped.apply(lambda group: (group['fechado'] == 1).sum())

In [ ]:
activity_df = pd.DataFrame({"Total years":comp_years,
                            "Active years":active_years, 
                            "Year no sales":years_no_sales,
                            "Inative years":inative_years})

activity_df

## 2012

In [ ]:
data_2012 = data[data['year'] == 2012].copy()
data_2012.year.unique()

### Sales

In [ ]:
data_2012.sales.describe()

Valores negativos não são válidos para a váriavel `sales`. O tratamento escolhido nesse caso é substituir valores negativos por 0.

In [ ]:
data_2012.loc[data['sales'] < 0] = 0
data_2012.sales.describe()

Podemos ver ainda nas estatísticas descritivas que a média é maior que o terceiro quartil, indicando uma distribuição bastante assimétrica. Criamos então uma com o logarítmo de `sales` para auxiliar na análise.

In [ ]:
data_2012.loc[:,'log_sales'] = data_2012.sales\
                                        .apply(lambda x: math.log(x)\
                                               if x != 0\
                                               else 0)

data_2012.log_sales.describe()

### Idade da Empresa
A idade da empresa eh calculada pela subtracao de `year` por `founded_year`, e a informacao eh guardada em `comp_age`.

In [ ]:
data.year.isna().sum()

In [ ]:
data.founded_year.isna().sum()

In [ ]:
duplicates = {'company': [], 'years': []}
nulls = {'company': [], 'years': []}

for company in data_2012.comp_id.unique():
    company_year = data_2012.loc[data_2012['comp_id'] == company,
                                 'founded_year'].unique()
    if len(company_year) != 1:
        duplicates['company'].append(company)
        duplicates['years'].append(company_year)
    elif np.nan in company_year:
        nulls['company'].append(company)
        nulls['years'].append(company_year)

nulls

In [ ]:
data_2012.loc[data_2012['comp_id'] == 1001034,
              'founded_year'].unique()

In [ ]:
data_2012.loc[(data_2012['comp_id'] == 1001034) &
              (data_2012['founded_year'] == np.nan)]

Criação de variáveis

In [ ]:
# 1. Idade da Empresa:
data_2012['company_age'] = 2012 - data_2012['founded_year']

In [ ]:
# 2. Taxa de Crescimento de Vendas:
# Taxa de crescimento: (valor_atual - valor_anterior) / valor_anterior
data_2012['sales_growth_rate'] = (data_2012['sales'] - data_2012['sales_x1']) / data_2012['sales_x1']

In [ ]:
# 3. Alavancagem Financeira:
data_2012['financial_leverage'] = data_2012['curr_liab'] / data_2012['share_eq']

In [ ]:
# 4. Liquidez:
data_2012['liquidity_ratio'] = data_2012['liq_assets'] / data_2012['curr_liab']

In [ ]:
# 5. Eficiência:
data_2012['efficiency'] = data_2012['sales'] / data_2012['labor_avg']

In [ ]:
# 6. Duração do CEO no Cargo:
data_2012['ceo_duration_years'] = data_2012['inoffice_days'] / 365.0

In [ ]:
# 7. Tamanho da Empresa (exemplo baseado em vendas):
sales_bins = [0, 1e6, 1e9, float('inf')]  # Exemplo de categorias: <1M, 1M-1B, >1B
labels = ['small', 'medium', 'large']
data_2012['company_size'] = pd.cut(data_2012['sales'], bins=sales_bins, labels=labels, right=False)

In [ ]:
# 8. Margem de Lucro:
data_2012['profit_margin'] = data_2012['profit_loss_year'] / data_2012['sales']

In [ ]:
data_2012.to_csv("data_2012")